search on dataset with 3 inputs , solve shakil only search by name


In [17]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

struct Contact {
  char id[50];
  char name[50];
  char number[50];
};

__device__ bool check(char* str1, char* str2, int len) {
  for(int i = 0; str1[i] != '\0'; i++) {
    int j = 0;
    while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
      j++;
    }
    if(j == len-1) {
      return true;
    }
  }
  return false;
}

__global__ void searchPhonebook(Contact* phonebook, int num_contacts, char* search_name, int name_length) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if(idx < num_contacts) {
    if(check(phonebook[idx].name, search_name, name_length)) {
      printf("%s %s %s\n", phonebook[idx].id, phonebook[idx].name, phonebook[idx].number);
    }
  }
}

int main(int argc, char* argv[]) {
  if(argc != 3) {
    cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
    return 1;
  }

  string search_name = argv[1];
  int num_threads = atoi(argv[2]);
  // Mount Google Drive and copy the location
  string file_name = "/content/drive/MyDrive/csv/parallel dataset/labtest_dataset1.txt";
  //string file_name = "phonebook1.txt";

  vector<Contact> phonebook;

  ifstream file(file_name);
  if(!file.is_open()) {
    cerr << "Error opening file: " << file_name << endl;
    return 1;
  }
  else {
    Contact contact;
    string line;
    while(getline(file, line)) {
      /* Format: "id","name","phone_number"
      int pos = line.find(",");
      strcpy(contact.id, line.substr(1, pos-2).c_str());
      line = line.substr(pos+1);
      pos = line.find(",");
      strcpy(contact.name, line.substr(1, pos-2).c_str());
      strcpy(contact.number, line.substr(pos+2, line.size()-pos-4).c_str());
      phonebook.push_back(contact);
      */

      // Format: "name","phone_number"
      int pos = line.find(",");
      // Extract name (without the quotes)
      strcpy(contact.name, line.substr(1, pos - 2).c_str());

      // Extract number (also without quotes)
      strcpy(contact.number, line.substr(pos + 2, line.size() - pos - 4).c_str());

      phonebook.push_back(contact);
    }
    file.close();
  }
  int num_contacts = phonebook.size();
  Contact* device_phonebook;
  cudaMalloc((void**)&device_phonebook, sizeof(Contact)*num_contacts);
  cudaMemcpy(device_phonebook, phonebook.data(), sizeof(Contact)*num_contacts, cudaMemcpyHostToDevice);

  int name_length = search_name.length() + 1;
  char* device_search_name;
  cudaMalloc((void**)&device_search_name, name_length);
  cudaMemcpy(device_search_name, search_name.c_str(), name_length, cudaMemcpyHostToDevice);

  for(int i = 0; i < num_contacts; i += num_threads) {
    int thread_count = min(num_contacts-i, num_threads);
    searchPhonebook<<<1, thread_count>>>(device_phonebook + i, thread_count, device_search_name, name_length);
    cudaDeviceSynchronize();
  }

  cudaFree(device_phonebook);
  cudaFree(device_search_name);

  return 0;
}



Overwriting search_phonebook.cu


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [13]:
# ✅ real → wall clock time
# This is the total elapsed time — from the moment you launched the command to when it exited.
# It includes everything: CPU time, GPU time, waiting, I/O, and even time spent doing nothing.

# So real 0m0.001s → 1 millisecond total runtime.

# ✅ user → user CPU time
# This is the amount of time your program spent actively executing on the CPU (not including time spent in kernel mode or on the GPU).
# For CUDA programs, this includes:

#     setting up memory,

#     launching kernels,

#     copying data,
#     but not the time spent by the GPU itself.

# So user 0m0.001s → ~1 millisecond of active CPU work.

# ✅ sys → system (kernel) CPU time
# This is the time spent inside the operating system kernel, handling system calls, device management, etc.
# For CUDA, this might include:

#     memory allocations (cudaMalloc),

#     synchronization (cudaDeviceSynchronize),

#     copying between host and device.

# So sys 0m0.000s → almost zero system-level time.

In [23]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook
!time ./search_phonebook NUSRAT 100 > output_shakil.txt


real	0m15.274s
user	0m10.589s
sys	0m4.297s


solve by asif, only search by name

In [5]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

struct Contact {
    char name[65];
    char phone_number[65];
};

string getInput(ifstream& file) {
    string ans;
    char c;
    int readSuru = 0;
    while (file.get(c)) {
        if (c == '\"') {
            if (readSuru == 1) break;
            readSuru = 1;
        } else {
            if (readSuru) {
                ans.push_back(c);
            }
        }
    }
    return ans;
}

__device__ bool check(char* str1, char* str2) {
    for (int i = 0; str1[i] != '\0'; i++) {
        int flag = 1;
        for (int j = 0; str2[j] != '\0'; j++) {
            if (str1[i + j] != str2[j]) {
                flag = 0;
                break;
            }
        }
        if (flag == 1) return true;
    }
    return false;
}

__global__ void myKernel(Contact* phoneBook, char* pat, int offset) {
    int threadNumber = threadIdx.x + offset;
    if (check(phoneBook[threadNumber].name, pat)) {
        printf("%s %s\n", phoneBook[threadNumber].name, phoneBook[threadNumber].phone_number);
    }
}

int main(int argc, char* argv[]) {
    if (argc < 3) {
        cout << "Usage: ./phonebook_search <search_name> <thread_limit>" << endl;
        return 1;
    }

    int threadLimit = atoi(argv[2]);

    // Use the correct file path here
    ifstream myfile("/content/drive/MyDrive/csv/parallel dataset/labtest_dataset1.txt");
    if (!myfile.is_open()) {
        cerr << "Failed to open file." << endl;
        return 1;
    }

    vector<Contact> phoneBook;
    int count = 0;

    while (myfile.peek() != EOF) {
        string id = getInput(myfile);  // ignore ID
        string name = getInput(myfile);
        string phoneNum = getInput(myfile);

        Contact c;
        strcpy(c.name, name.c_str());
        strcpy(c.phone_number, phoneNum.c_str());

        phoneBook.push_back(c);
        count++;
    }

    string search_name = argv[1];
    char pat[65];
    strcpy(pat, search_name.c_str());

    char* d_pat;
    cudaMalloc(&d_pat, 65);
    cudaMemcpy(d_pat, pat, 65, cudaMemcpyHostToDevice);

    int n = phoneBook.size();
    Contact* d_phoneBook;
    cudaMalloc(&d_phoneBook, n * sizeof(Contact));
    cudaMemcpy(d_phoneBook, phoneBook.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);

    int bakiAche = n;
    int offset = 0;
    while (bakiAche > 0) {
        int batchSize = min(threadLimit, bakiAche);
        myKernel<<<1, batchSize>>>(d_phoneBook, d_pat, offset);
        cudaDeviceSynchronize();

        bakiAche -= batchSize;
        offset += batchSize;
    }

    cudaFree(d_phoneBook);
    cudaFree(d_pat);

    return 0;
}


Overwriting phonebook_search.cu


In [6]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [11]:
!time ./phonebook_search 015 1 > output.txt


real	2m20.334s
user	2m16.453s
sys	0m2.814s


In [8]:
!time ./phonebook_search MUMU 2 > output.txt


real	1m13.478s
user	1m10.642s
sys	0m2.242s


In [9]:
!time ./phonebook_search MUMU 10 > output.txt


real	0m24.773s
user	0m22.182s
sys	0m2.328s


In [ ]:
%%writefile asif.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;

__global__ void matrixMul(float* A, float* B, float* C, int M, int N, int P, int offset) {
    int k = threadIdx.x + offset;

    float* a = A + k * M * N;
    float* b = B + k * N * P;
    float* c = C + k * M * P;

    for(int i = 0; i < M; i++) {
        for(int j = 0; j < N; j++) {
            for(int l = 0; l < P; l++) {
                //c[i][l] += a[i][j] * b[j][l];
                c[i * P + l] = a[i * N + j] * b[j * P + l];
            }
        }
    }
}

int main(int argc, char *argv[]) {

    int T = atoi(argv[1]); //koyta thread use korte parbo
    int K = atoi(argv[2]); //koita matrix gun

    //100 gun, thread 10,

    int M = 400, N = 400, P = 400;

    int SizeA = M * N * K;
    int SizeB = N * P * K;
    int SizeC = M * P * K;

    //memory alocate (cpu allocate)
    float *h_A = new float[SizeA];
    float *h_B = new float[SizeB];
    float *h_C = new float[SizeC];


    //malloc (gpu allocate)
    float *d_A;
    cudaMalloc(&d_A, SizeA * sizeof(float));
    float *d_B;
    cudaMalloc(&d_B, SizeB * sizeof(float));
    float *d_C;
    cudaMalloc(&d_C, SizeC * sizeof(float));

    //data initialize
    for (int i = 0; i < SizeA; i++) {
        h_A[i] = rand();
    }
    for(int i = 0; i < SizeB; i++) {
        h_B[i] = rand();
    }


    //copy from host to device
    cudaMemcpy(d_A, h_A, SizeA * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, SizeB * sizeof(float), cudaMemcpyHostToDevice);

    //cuda process suru
    int gunKorteHobe = K;
    int offset = 0;
    while(gunKorteHobe > 0){

        int currentBatch = min(gunKorteHobe, T);

        matrixMul<<<1,currentBatch>>>(d_A, d_B, d_C, M, N, P, offset);
        cudaDeviceSynchronize();

        gunKorteHobe -= currentBatch;
        offset += currentBatch;
    }

    //let's copy back to cpu
    cudaMemcpy(h_C, d_C, SizeC * sizeof(float), cudaMemcpyDeviceToHost);

    cout << "All operation done" << endl;

}

Writing asif.cu


In [ ]:
!nvcc -arch=sm_75 asif.cu -o asif

In [ ]:
!time ./asif 3 10 && sleep 2

All operation done

real	0m12.917s
user	0m12.607s
sys	0m0.215s


In [ ]:
!time ./asif 3 10 && sleep 2

All operation done

real	0m12.920s
user	0m12.578s
sys	0m0.214s


In [ ]:
!time ./asif 20 10 && sleep 2

All operation done

real	0m5.227s
user	0m4.931s
sys	0m0.203s


In [ ]:
#include <bits/stdc++.h>
#include <mpi.h>
using namespace std;

struct Contact {
    string name;
    string phone;
};

void send_string(const string &text, int receiver) {
    int len = text.size() + 1;
    MPI_Send(&len, 1, MPI_INT, receiver, 1, MPI_COMM_WORLD);
    MPI_Send(text.c_str(), len, MPI_CHAR, receiver, 1, MPI_COMM_WORLD);
}

string receive_string(int sender) {
    int len;
    MPI_Recv(&len, 1, MPI_INT, sender, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    char *buf = new char[len];
    MPI_Recv(buf, len, MPI_CHAR, sender, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    string res(buf);
    delete[] buf;
    return res;
}

string vector_to_string(const vector<Contact> &contacts, int start, int end) {
    string result;
    for (int i = start; i < min((int)contacts.size(), end); i++) {
        result += contacts[i].name + "," + contacts[i].phone + "\n";
    }
    return result;
}

vector<Contact> string_to_contacts(const string &text) {
    vector<Contact> contacts;
    istringstream iss(text);
    string line;
    while (getline(iss, line)) {
        if (line.empty()) continue;
        int comma = line.find(",");
        if (comma == string::npos) continue;
        contacts.push_back({line.substr(0, comma), line.substr(comma + 1)});
    }
    return contacts;
}

string check(const Contact &c, const string &search) {
    if (c.name.find(search) != string::npos) {
        return c.name + " " + c.phone + "\n";
    }
    return "";
}

void read_phonebook(const vector<string> &files, vector<Contact> &contacts) {
    for (const string &file : files) {
        ifstream f(file);
        string line;
        while (getline(f, line)) {
            if (line.empty()) continue;
            int comma = line.find(",");
            if (comma == string::npos) continue;
            contacts.push_back({line.substr(1, comma - 2), line.substr(comma + 2, line.size() - comma - 3)});
        }
    }
}

int main(int argc, char **argv) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    if (argc < 3) {
        if (rank == 0)
            cerr << "Usage: mpirun -n <procs> " << argv[0] << " <file>... <search>\n";
        MPI_Finalize();
        return 1;
    }

    string search_term = argv[argc - 1];
    double start, end;

    if (rank == 0) {
        vector<string> files(argv + 1, argv + argc - 1);
        vector<Contact> contacts;
        read_phonebook(files, contacts);
        int total = contacts.size();
        int chunk = (total + size - 1) / size;

        for (int i = 1; i < size; i++) {
            string text = vector_to_string(contacts, i * chunk, (i + 1) * chunk);
            send_string(text, i);
        }

        start = MPI_Wtime();
        string result;
        for (int i = 0; i < min(chunk, total); i++) {
            string match = check(contacts[i], search_term);
            if (!match.empty()) result += match;
        }
        end = MPI_Wtime();

        for (int i = 1; i < size; i++) {
            string recv = receive_string(i);
            if (!recv.empty()) result += recv;
        }

        ofstream out("output.txt");
        out << result;
        out.close();
        printf("Process %d took %f seconds.\n", rank, end - start);

    } else {
        string recv_text = receive_string(0);
        vector<Contact> contacts = string_to_contacts(recv_text);
        start = MPI_Wtime();
        string result;
        for (auto &c : contacts) {
            string match = check(c, search_term);
            if (!match.empty()) result += match;
        }
        end = MPI_Wtime();
        send_string(result, 0);
        printf("Process %d took %f seconds.\n", rank, end - start);
    }

    MPI_Finalize();
    return 0;
}


In [ ]:
 mpic++ -o search phonebook_search.cpp
 mpirun -np 3 ./search phoneboo1.txt 015

In [ ]:
#include <bits/stdc++.h>
#include <mpi.h>
using namespace std;

struct Contact {
    string id;
    string name;
    string phone;
};

void send_string(const string &text, int receiver) {
    int len = text.size() + 1;
    MPI_Send(&len, 1, MPI_INT, receiver, 1, MPI_COMM_WORLD);
    MPI_Send(text.c_str(), len, MPI_CHAR, receiver, 1, MPI_COMM_WORLD);
}

string receive_string(int sender) {
    int len;
    MPI_Recv(&len, 1, MPI_INT, sender, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    char *buf = new char[len];
    MPI_Recv(buf, len, MPI_CHAR, sender, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    string res(buf);
    delete[] buf;
    return res;
}

string vector_to_string(const vector<Contact> &contacts, int start, int end) {
    string result;
    for (int i = start; i < min((int)contacts.size(), end); i++) {
        result += contacts[i].id + "," + contacts[i].name + "," + contacts[i].phone + "\n";
    }
    return result;
}

vector<Contact> string_to_contacts(const string &text) {
    vector<Contact> contacts;
    istringstream iss(text);
    string line;
    while (getline(iss, line)) {
        if (line.empty()) continue;
        vector<string> parts;
        istringstream ls(line);
        string part;
        while (getline(ls, part, ',')) {
            parts.push_back(part);
        }
        if (parts.size() == 3) {
            contacts.push_back({parts[0], parts[1], parts[2]});
        }
    }
    return contacts;
}

string check(const Contact &c, const string &search) {
    if (c.name.find(search) != string::npos) {
        return c.id + " " + c.name + " " + c.phone + "\n";
    }
    return "";
}

void read_phonebook(const vector<string> &files, vector<Contact> &contacts) {
    for (const string &file : files) {
        ifstream f(file);
        string line;
        while (getline(f, line)) {
            if (line.empty()) continue;
            vector<string> parts;
            istringstream ls(line);
            string part;
            while (getline(ls, part, ',')) {
                parts.push_back(part);
            }
            if (parts.size() == 3) {
                contacts.push_back({parts[0], parts[1], parts[2]});
            }
        }
    }
}

int main(int argc, char **argv) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    if (argc < 3) {
        if (rank == 0)
            cerr << "Usage: mpirun -n <procs> " << argv[0] << " <file>... <search>\n";
        MPI_Finalize();
        return 1;
    }

    string search_term = argv[argc - 1];
    double start, end;

    if (rank == 0) {
        vector<string> files(argv + 1, argv + argc - 1);
        vector<Contact> contacts;
        read_phonebook(files, contacts);
        int total = contacts.size();
        int chunk = (total + size - 1) / size;

        for (int i = 1; i < size; i++) {
            string text = vector_to_string(contacts, i * chunk, (i + 1) * chunk);
            send_string(text, i);
        }

        start = MPI_Wtime();
        string result;
        for (int i = 0; i < min(chunk, total); i++) {
            string match = check(contacts[i], search_term);
            if (!match.empty()) result += match;
        }
        end = MPI_Wtime();

        for (int i = 1; i < size; i++) {
            string recv = receive_string(i);
            if (!recv.empty()) result += recv;
        }

        ofstream out("output2.txt");
        out << result;
        out.close();
        printf("Process %d took %f seconds.\n", rank, end - start);

    } else {
        string recv_text = receive_string(0);
        vector<Contact> contacts = string_to_contacts(recv_text);
        start = MPI_Wtime();
        string result;
        for (auto &c : contacts) {
            string match = check(c, search_term);
            if (!match.empty()) result += match;
        }
        end = MPI_Wtime();
        send_string(result, 0);
        printf("Process %d took %f seconds.\n", rank, end - start);
    }

    MPI_Finalize();
    return 0;
}


Overwriting search_phonebook.cu


In [ ]:
 mpic++ -o search phonebook_search.cpp
 mpirun -np 3 ./search phoneboo1.txt 015

In [ ]:
time ./search_phonebook SHAKIL 100 > output1.txt

SyntaxError: invalid syntax (<unknown>, line 1)